In [1]:
import numpy as np
import pandas as pd
from pandas.tseries.offsets import DateOffset
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
comex_df=pd.read_csv('/home/andre301267/git/Pricing-Fertilizer/DB/temp/imp_fert_ncm_quanti.csv')

In [2]:
comex_df=comex_df[(comex_df.Product.isin(['MOP','MAP','SAM','UREIA']))&(comex_df.UF.isin(['BA']))]

In [3]:
filtra_km=comex_df[['Product','km']].drop_duplicates().sort_values(by=['Product','km'],ascending=False).drop_duplicates(subset='Product')
comex_df=comex_df.merge(filtra_km)

In [4]:
comex_df['date']=pd.to_datetime(comex_df.year.astype(str)+'-'+comex_df.month.astype(str)+'-01')
date_min=comex_df.date.min()
date_max=comex_df.date.max()

In [5]:
years=pd.DataFrame({'year':np.arange(comex_df.year.min(),comex_df.year.max()+1)}).assign(key=0)
months=pd.DataFrame({'month':np.arange(1,13)}).assign(key=0)
calendar=years.merge(months,on='key')
full=comex_df[['Product','UF']].drop_duplicates().assign(key=0).merge(calendar,on='key').drop('key',axis=1)
comex_df=comex_df.merge(full)

In [6]:
round(comex_df[comex_df.Product=='MOP'].kg.sum()/10**9)

8

In [7]:
comex_df=comex_df.groupby(by=['year','month','Product'],observed=False).kg.sum().reset_index()

In [8]:
comex_df['date']=pd.to_datetime(comex_df.year.astype(str)+'-'+comex_df.month.astype(str)+'-01')

In [9]:
comex_df=comex_df[(comex_df.date>=date_min)&(comex_df.date<=date_max)]

In [10]:
comex_df[['month','Product']]=comex_df[['month','Product']].astype(str)

In [11]:
comex_df=pd.get_dummies(comex_df)

In [12]:
comex_df=comex_df.assign(oc=0)
comex_df.loc[comex_df.kg>0,'oc']=1

# Classify

In [13]:
comex_df.shape

(836, 20)

In [14]:
for r in range(1,6):
    train=comex_df[comex_df.date<comex_df.date.max()-DateOffset(months=r*12-1)]
    test=comex_df[comex_df.date==comex_df.date.max()-DateOffset(months=r*12-1)]
    X_train=train.drop(['kg','oc','date'],axis=1)
    y_train=train.oc
    X_test=test.drop(['kg','oc','date'],axis=1)
    y_test=test.oc
    from sklearn.ensemble import RandomForestClassifier as rfc
    model=rfc()
    model.fit(X_train,y_train)
    y_pred=model.predict(X_test)
    test['y_pred']=y_pred
    test['y_test']=y_test
    test['r']=r
    if r==1:
        results=test
    else:
        results=pd.concat([results,test])

In [15]:
comex_df=results.merge(comex_df,how='outer')

In [16]:
comex_df.shape

(836, 23)

In [17]:
comex_df=comex_df[(comex_df.oc==1)&(comex_df.y_pred.isin([1,np.nan]))]

In [18]:
comex_df.shape

(836, 23)

# Regression

In [21]:
features_df=pd.DataFrame()
for r in range(1,11):
    train=comex_df[comex_df.date<comex_df.date.max()-DateOffset(months=r*12-1)]
    test=comex_df[comex_df.date==comex_df.date.max()-DateOffset(months=r*12-1)]
    X_train=train.drop(['kg','oc','date'],axis=1)
    y_train=train.kg
    X_test=test.drop(['kg','oc','date'],axis=1)
    y_test=test.kg
    from sklearn.ensemble import RandomForestRegressor as rfr
    model=rfr(max_features=5)
    model.fit(X_train,y_train)
    t=pd.DataFrame({'names':model.feature_names_in_,'importances':model.feature_importances_})
    features_df=pd.concat([features_df,t])
    y_pred=model.predict(X_test)
    test['y_pred']=y_pred
    test['y_test']=y_test
    test['r']=r
    if r==1:
        results=test
    else:
        results=pd.concat([results,test])


display(features_df.groupby('names').importances.mean().reset_index().sort_values(by='importances',ascending=False))

,names,importances
19,year,0.503577
1,Product_MOP,0.123444
2,Product_SAM,0.097184
0,Product_MAP,0.028232
13,month_7,0.026613
15,month_9,0.025660
3,Product_UREIA,0.023463
14,month_8,0.021041
6,month_11,0.020944
5,month_10,0.019455


In [22]:
results=results[results.Product_MOP==1]

In [23]:
results[['date','kg','y_test','y_pred']]

,date,kg,y_test,y_pred
802,2023-09-01,7.377500e+07,7.377500e+07,5.298534e+07
754,2022-09-01,5.513027e+07,5.513027e+07,5.500965e+07
709,2021-09-01,4.308353e+07,4.308353e+07,4.712157e+07
664,2020-09-01,5.204514e+07,5.204514e+07,4.928896e+07
609,2019-09-01,2.760000e+07,2.760000e+07,8.221732e+07
580,2018-09-01,1.180357e+08,1.180357e+08,4.288023e+07
539,2017-09-01,5.232994e+07,5.232994e+07,3.899663e+07
500,2016-09-01,2.483312e+07,2.483312e+07,5.893895e+07
481,2015-09-01,8.410621e+07,8.410621e+07,3.139535e+07
441,2014-09-01,3.091048e+07,3.091048e+07,3.451507e+07


In [24]:
from sklearn.metrics import root_mean_squared_error as rse
results['rse']=rse(results.y_test,results.y_pred)

In [25]:
int(results.rse.unique())/10**6

36.359222